## Transfer Learning using MNIST data
To illustrate the power and concept of transfer learning, we will train a CNN on just the digits 5,6,7,8,9.  Then we will train just the last layer(s) of the network on the digits 0,1,2,3,4 and see how well the features learned on 5-9 help with classifying 0-4.



In [52]:
import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [53]:
#used to help some of the timing functions
now = datetime.datetime.now

In [54]:
# set some parameters
batch_size = 128
num_classes = 5
epochs = 5

In [55]:
# set some more parameters
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

In [56]:
## To simplify things, write a function to include all the training steps
## As input, function takes a model, training set, test set, and the number of classes
## Inside the model object will be the state about which layers we are freezing and which we are training
#Reshape the data
#Normalize the data
# One hot encode the targert label
# Compile the model
# Train the model on the training data
# Evaluate the model on the testing data

# Check image data format
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

def  train_model (model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0]. reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype( 'float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train. shape)
    print(x_train.shape[0], 'train samples') 
    print(x_test.shape[0], 'test samples')
    
    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils. to_categorical(test[1], num_classes)
    model. compile(loss= 'categorical_crossentropy',
                   optimizer='adadelta', metrics=['accuracy'])
    
    t = now()
    model. fit(x_train, y_train, batch_size=batch_size,
               epochs=epochs,
               verbose=1, 
               validation_data=(x_test, y_test))
    print( 'Training time: %s' % (now() - t))
    
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])


In [57]:
# Load the Mnist data and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Create two datasets: one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

from keras import backend as K
# Set some more parameters
img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)


In [58]:
# Define the "feature" layers. Add 2 convolution layer with max pool layer. At the end, add dropout layer with 0.25% probability and end with the flatten layer. These are the early layers that we expect will "transfer"
# to a new problem.  We will freeze these layers during the fine-tuning process

feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

In [59]:
# Define the "classification" layers. Add Dense layer with 128 nodes and the output dense layer. These are the later layers that predict the specific classes from the features
# learned by the feature layers.  This is the part of the model that needs to be re-trained for a new problem

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')

]

In [60]:
# Create the model by combining the two sets of layers:
model = Sequential(feature_layers + classification_layers)

In [61]:
# print model summary
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 26, 26, 32)        │        320 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ activation_14 (Activation)      │ (None, 26, 26, 32)        │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_9 (Conv2D)               │ (None, 24, 24, 32)        │      9,248 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ activation_15 (Activation)      │ (None, 24, 24, 32)        │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 12, 12, 32)        │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_7 (Dropout)             │ (None, 12, 12, 32)        │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ flatten_4 (Flatten)             │ (None, 4608)              │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_6 (Dense)                 │ (None, 128)               │    589,952 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ activation_16 (Activation)      │ (None, 128)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_8 (Dropout)             │ (None, 128)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_7 (Dense)                 │ (None, 5)                 │        645 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ activation_17 (Activation)      │ (None, 5)                 │          0 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 600,165 (2.29 MB)

 Trainable params: 600,165 (2.29 MB)

 Non-trainable params: 0 (0.00 B)

In [62]:
# Now, let's train our model on the digits 5,6,7,8,9
train_model(model, (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)


x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 38s 146ms/step - accuracy: 0.1962 - loss: 1.6080 - val_accuracy: 0.3327 - val_loss: 1.5810
Epoch 2/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 35s 145ms/step - accuracy: 0.2670 - loss: 1.5809 - val_accuracy: 0.4787 - val_loss: 1.5507
Epoch 3/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 42s 149ms/step - accuracy: 0.3358 - loss: 1.5531 - val_accuracy: 0.5816 - val_loss: 1.5171
Epoch 4/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 35s 147ms/step - accuracy: 0.4054 - loss: 1.5225 - val_accuracy: 0.6725 - val_loss: 1.4785
Epoch 5/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 41s 148ms/step - accuracy: 0.4739 - loss: 1.4847 - val_accuracy: 0.7562 - val_loss: 1.4329
Training time: 0:03:11.385275
Test score: 1.4328389167785645
Test accuracy: 0.7561782598495483


### Freezing Layers
Keras allows layers to be "frozen" during the training process.  That is, some layers would have their weights updated during the training process, while others would not.  This is a core part of transfer learning, the ability to train just the last one or several layers.

Note also, that a lot of the training time is spent "back-propagating" the gradients back to the first layer.  Therefore, if we only need to compute the gradients back a small number of layers, the training time is much quicker per iteration.  This is in addition to the savings gained by being able to train on a smaller data set.

In [63]:
# Freeze only the feature layers
for i in feature_layers:
    i.trainable = False

Observe below the differences between the number of *total params*, *trainable params*, and *non-trainable params*.

In [64]:
# print model summary
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 26, 26, 32)        │        320 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ activation_14 (Activation)      │ (None, 26, 26, 32)        │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_9 (Conv2D)               │ (None, 24, 24, 32)        │      9,248 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ activation_15 (Activation)      │ (None, 24, 24, 32)        │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 12, 12, 32)        │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_7 (Dropout)             │ (None, 12, 12, 32)        │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ flatten_4 (Flatten)             │ (None, 4608)              │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_6 (Dense)                 │ (None, 128)               │    589,952 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ activation_16 (Activation)      │ (None, 128)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_8 (Dropout)             │ (None, 128)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_7 (Dense)                 │ (None, 5)                 │        645 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ activation_17 (Activation)      │ (None, 5)                 │          0 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 1,800,497 (6.87 MB)

 Trainable params: 590,597 (2.25 MB)

 Non-trainable params: 9,568 (37.38 KB)

 Optimizer params: 1,200,332 (4.58 MB)

In [65]:
# Now, let's train our model on the digits 0,1,2,3,4
train_model(model, (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 17s 66ms/step - accuracy: 0.5335 - loss: 1.4453 - val_accuracy: 0.8040 - val_loss: 1.4004
Epoch 2/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 14s 58ms/step - accuracy: 0.5784 - loss: 1.4150 - val_accuracy: 0.8328 - val_loss: 1.3677
Epoch 3/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 14s 60ms/step - accuracy: 0.6194 - loss: 1.3850 - val_accuracy: 0.8533 - val_loss: 1.3351
Epoch 4/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 14s 59ms/step - accuracy: 0.6523 - loss: 1.3569 - val_accuracy: 0.8677 - val_loss: 1.3028
Epoch 5/5
240/240 ━━━━━━━━━━━━━━━━━━━━ 14s 58ms/step - accuracy: 0.6899 - loss: 1.3222 - val_accuracy: 0.8795 - val_loss: 1.2708
Training time: 0:01:13.929839
Test score: 1.270560622215271
Test accuracy: 0.8795485496520996


Note that after a single epoch, we are already achieving results on classifying 0-4 that are comparable to those achieved on 5-9 after 5 full epochs.  This despite the fact the we are only "fine-tuning" the last layer of the network, and all the early layers have never seen what the digits 0-4 look like.

Also, note that even though nearly all (590K/600K) of the *parameters* were trainable, the training time per epoch was still much reduced.  This is because the unfrozen part of the network was very shallow, making backpropagation faster. 